In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [3]:
from factue.methods.llm_calls import load_template_list, load_template_parts
for x in load_template_list(job='persuasion', step="explain", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Guilt_by_Association"
"Doubt"
"Appeal_to_Hypocrisy"
"Questioning_the_Reputation"
"Flag_Waving"
"Appeal_to_Authority"
"Appeal_to_Popularity"
"Appeal_to_Values"
"Appeal_to_Fear-Prejudice"
"Straw_Man"
"Red_Herring"
"Whataboutism"
"Appeal_to_Pity"
"Causal_Oversimplification"
"False_Dilemma-No_Choice"
"Consequential_Oversimplification"
"False_Equivalence"
"Slogans"
"Conversation_Killer"
"Appeal_to_Time"
"Loaded_Language"
"Obfuscation-Vagueness-Confusion"
"Exaggeration-Minimisation"
"Repetition"


In [4]:
import pandas as pd
from pathlib import Path
import os
import json 
import numpy as np

root = Path("data/llm_output/persuasion/explain")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

def safe_description(data):
    if isinstance(data, (list,np.ndarray)):
        return data[-1]
    else:
        return None  # or "", depending on what you prefer
    
df['expected_output'] = df.apply(lambda row: json.dumps({
    'description': safe_description(row['explanation']),
    'verdict': bool(row['gold'])
}), axis=1)
df

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,updated_definition,think_content,illegal_value,explanation,extra_content,source_file,error,status,split,expected_output
0,pl_eu_12_06_2024_n01.txt,0,135,PL,Wicemarszałek Włodzimierz Czarzasty:\nBardzo p...,False,[],train,[],[],...,[None],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,pl_defence_22_05_2024_n01.txt,3608,3857,PL,Pan wspominał o ABCS – Integrated Battle Comma...,False,[],train,[],[],...,[],[None],[None],[The input text does not contain any instances...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
2,pl_current_affairs_09_01_2025_n03.txt,1808,1984,PL,(Poseł Anna Gembicka: Decyzja została zmienion...,True,[Repetition],train,[],[],...,[],[None],[None],[The Exaggeration - Minimisation technique was...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The Exaggeration - Minimisati..."
3,pl_abortion_11_04_2024_n08.txt,2952,3136,PL,"Czy czas najwyższy, by w końcu dopuścić\ndekry...",True,"[False_Equivalence, Loaded_Language, Whatabout...",train,[],[],...,[],[None],[None],[The input text does not employ the Exaggerati...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
4,pl_eu_12_06_2024_n01.txt,1277,2304,PL,"Prezentowany na posiedzeniu projekt ustawy, op...",False,[],train,[],[],...,[],[None],[None],[The input text does not employ the Exaggerati...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15486,20231208_URW_BG_71.txt,3950,3965,BG,Има ли реплика?,False,[],train,[],[],...,[],[None],[None],[The input text does not contain any arguments...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
15487,20231208_URW_BG_76.txt,1541,1938,BG,"Най-интересното е, че в т. 7 пише, цитирам, че...",False,[],train,[],[],...,[None],[None],[None],[The input text does not employ the Strawman t...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
15488,20240704_URW_BG_25.txt,4841,5237,BG,"и, предлагам това заседание, госпожо Председат...",True,"[Appeal_to_Hypocrisy, Loaded_Language]",train,[],[],...,[],[None],[None],[The input text does not demonstrate the Straw...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not demon..."
15489,20240704_URW_BG_25.txt,1179,1237,BG,"АВ РИБАРСКИ (ПП-ДБ): Благодаря, госпожо Предсе...",False,[],train,[],[],...,[],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."


In [5]:
df[df["is_valid"].apply(
    lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [False]
)]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,updated_definition,think_content,illegal_value,explanation,extra_content,source_file,error,status,split,expected_output
11106,pl_abortion_11_04_2024_n03.txt,4168,5204,PL,"W latach 90., które już bardzo dobrze pamiętam...",True,"[Appeal_to_Hypocrisy, Loaded_Language, Questio...",train,[ustawa była początkiem wymazywania kobiet z d...,[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
11284,SI_interpelation_interior_12.txt,4448,7979,SI,Najlepša hvala za besedo. Najprej hvala gospod...,True,"[Consequential_Oversimplification, Exaggeratio...",train,"[tisti, ki vam je pisal odgovore na našo inter...",[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
11292,SI_interpelation_interior_7.txt,2598,5288,SI,"Policisti prav tako dobijo navodila, naj ne sp...",True,"[Consequential_Oversimplification, Exaggeratio...",train,[Taka navodila policistom seveda preprečujejo ...,[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
11310,SI_interpelation_interior_7.txt,839,2596,SI,Naslednji primer: žena pomočnika generalnega d...,True,"[Causal_Oversimplification, Exaggeration-Minim...",train,[mora po navodilu pomočnika generalnega direkt...,[golobistov],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
11317,SI_interpelation_interior_6.txt,270,7305,SI,Na eni izmed sej Odbora za notranje zadeve mi ...,True,"[Loaded_Language, Obfuscation-Vagueness-Confus...",train,"[Še posebej direktorja, tam na mojem koncu, ko...","[golobisti, janšistov, janšisti, janšistov, go...",...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,20240704_URW_BG_30.txt,2946,2988,BG,"Нинова, за процедура.\n\nИзчакайте, господи",False,[],train,[],[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"
14694,20231208_URW_BG_72.txt,1795,2118,BG,"Говорите, че машините, които гласувахме – „Стр...",True,"[Questioning_the_Reputation, Doubt]",train,"[Ако за Вас 1956 г. е същата като 2001 г., за ...",[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
14695,20240229_URW_BG_67.txt,2786,4165,BG,"И да Ви кажа, уважаеми български граждани, че ...",True,"[Appeal_to_Hypocrisy, Questioning_the_Reputati...",train,"[И да Ви кажа, уважаеми български граждани, че...",[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
14726,20240704_URW_BG_25.txt,4080,4133,BG,оспожо Председател.\n\nЩе направя процедурно п...,False,[],train,[],[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"


In [6]:
clean = df[~df["explanation"].apply(
    lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [None]
)]
clean.prompt_name.value_counts()

prompt_name
Exaggeration-Minimisation           692
Appeal_to_Time                      692
Whataboutism                        692
Slogans                             691
False_Equivalence                   691
Flag_Waving                         691
Obfuscation-Vagueness-Confusion     691
Loaded_Language                     690
Consequential_Oversimplification    690
Repetition                          689
False_Dilemma-No_Choice             689
Appeal_to_Pity                      689
Guilt_by_Association                688
Straw_Man                           688
Red_Herring                         687
Causal_Oversimplification           687
Conversation_Killer                 685
Appeal_to_Hypocrisy                 677
Name_Calling-Labeling               639
Appeal_to_Fear-Prejudice            371
Appeal_to_Authority                 370
Appeal_to_Popularity                369
Appeal_to_Values                    369
Doubt                               350
Questioning_the_Reputation  

In [7]:
clean.value_counts(['prompt_name','gold'])

prompt_name                       gold
False_Equivalence                 0       681
Guilt_by_Association              0       676
Appeal_to_Time                    0       674
Whataboutism                      0       673
Straw_Man                         0       670
Slogans                           0       670
False_Dilemma-No_Choice           0       666
Appeal_to_Pity                    0       663
Obfuscation-Vagueness-Confusion   0       663
Flag_Waving                       0       662
Consequential_Oversimplification  0       660
Red_Herring                       0       659
Causal_Oversimplification         0       647
Conversation_Killer               0       642
Appeal_to_Hypocrisy               0       636
Exaggeration-Minimisation         0       621
Repetition                        0       620
Name_Calling-Labeling             0       582
Loaded_Language                   0       580
Appeal_to_Popularity              0       344
Appeal_to_Fear-Prejudice          0      

In [8]:
# Assume your DataFrame is called df
label_1 = clean[clean['gold'] == 1]

# Count how many label==1 per category
counts = label_1['prompt_name'].value_counts()

# For label == 0, sample same number of rows per category
label_0_samples = (
    df[df['gold'] == 0]
    .groupby('prompt_name', group_keys=False)
    .apply(lambda x: x.sample(n=3*counts.get(x.name, 0)))
)

# Combine both
balanced_df31 = pd.concat([label_1, label_0_samples])
balanced_df31 = balanced_df31.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df31

/var/folders/lp/5z8ys0rj1q960h2656xp8c680000gn/T/ipykernel_49770/905121920.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=3*counts.get(x.name, 0)))


,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,updated_definition,think_content,illegal_value,explanation,extra_content,source_file,error,status,split,expected_output
0,RU_demography_3.txt,0,108,RU,"Вымирание ускоряется. Вышла статистика ""прорыв...",True,[Appeal_to_Time],train,[],[],...,"[The speaker uses the same word, phrase, story...",[None],[None],[The input text does not contain any repeated ...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,pl_abortion_11_04_2024_n04.txt,7726,7807,PL,"Przepraszam, że dopiero teraz, ale emocje,\npr...",False,[],train,[],[],...,[None],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
2,RU_discrediting_the_west_11.txt,129,594,RU,"евой генерал, результативный, не мясник - не н...",True,"[Loaded_Language, Obfuscation-Vagueness-Confus...",train,[],"[коррупционеры, которые отсиживаются в доброво...",...,[Loaded Language is the use of specific words ...,[None],[None],[The input text employs Loaded Language throug...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text employs Loaded..."
3,20240704_URW_BG_30.txt,5823,6119,BG,"заседанието, което ще бъде абсолютно безсмисле...",True,"[Consequential_Oversimplification, Exaggeratio...",train,[],[абсолютно безсмислено],...,[],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
4,RU_migrants_4.txt,836,1171,RU,"И еще один, тоже очень важный аспект. Родители...",True,[Questioning_the_Reputation],train,"[Среди детей мигрантов, в том числе подростков...",[],...,[],[None],[None],[The input text does not employ the Exaggerati...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,RU_discrediting_the_west_4.txt,244,418,RU,А там намечается самая радикальная перестройка...,False,[],train,[],[],...,[None],[None],[None],[The input text discusses a significant restru...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text discusses a si..."
3736,RU_URW_2.txt,2035,2295,RU,Наша партия в конце февраля проводит Всероссий...,False,[],train,[],[],...,[None],[None],[None],[The input text does not evoke feelings of pit...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not evoke..."
3737,20231208_URW_BG_71.txt,973,1687,BG,Идеята да отхвърлим ветото на президента е кра...,True,"[Exaggeration-Minimisation, Flag_Waving, Quest...",train,"[Нямам съмнение, че евро-атлантическото мнозин...",[евро-атлантическото мнозинство],...,[Name Calling - Labeling is a form of argument...,[None],[None],[The Name Calling - Labeling technique is used...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The Name Calling - Labeling t..."
3738,pl_defence_22_05_2024_n02.txt,1232,1679,PL,"Produkujemy śladowe ilości amunicji, 30 tys. s...",True,"[Causal_Oversimplification, Red_Herring]",train,[],[],...,"[The speaker uses the same word, phrase, story...",[None],[None],[The Repetition technique was not used in the ...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The Repetition technique was ..."


In [9]:
# Assume your DataFrame is called df
label_1 = clean[clean['gold'] == 1]

# Count how many label==1 per category
counts = label_1['prompt_name'].value_counts()

# For label == 0, sample same number of rows per category
label_0_samples = (
    df[df['gold'] == 0]
    .groupby('prompt_name', group_keys=False)
    .apply(lambda x: x.sample(n=2*counts.get(x.name, 0)))
)

# Combine both
balanced_df21 = pd.concat([label_1, label_0_samples])
balanced_df21 = balanced_df21.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df21

/var/folders/lp/5z8ys0rj1q960h2656xp8c680000gn/T/ipykernel_49770/374312229.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=2*counts.get(x.name, 0)))


,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,updated_definition,think_content,illegal_value,explanation,extra_content,source_file,error,status,split,expected_output
0,20240229_URW_BG_67.txt,4561,5284,BG,ХРИСТО ГАДЖЕВ (ГЕРБ-СДС): Уважаеми господин Пр...,False,[],train,[],[],...,[],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,20240704_URW_BG_28.txt,8169,8224,BG,.\n\nВИКТОРИЯ ВАСИЛЕВА (ВЕЛИЧИЕ): Благодаря Ви...,False,[],train,[],[],...,[None],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
2,pl_abortion_11_04_2024_n04.txt,5601,6150,PL,"Dzisiaj prezentujemy też ustawę, która odwraca...",True,"[Conversation_Killer, Appeal_to_Pity, Slogans,...",train,[],[bulwersujący wyrok],...,[],[None],[None],[The input text does not employ the Consequent...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
3,pl_eu_12_06_2024_n01.txt,4530,5548,PL,"Natomiast to, co jest kłopotem przedłożonego p...",True,[Doubt],train,[],[],...,[],[None],[None],[The input text discusses concerns about a leg...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text discusses conc..."
4,pl_defence_22_05_2024_n02.txt,1232,1679,PL,"Produkujemy śladowe ilości amunicji, 30 tys. s...",True,"[Causal_Oversimplification, Red_Herring]",train,[],[],...,[None],[None],[None],[The input text does not employ the Consequent...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,SI_interpelation_interior_2.txt,5949,8216,SI,"Pred skoraj dvema letoma, natančneje 21. febru...",True,[Repetition],train,[],[],...,[],[None],[None],[The input text does not employ the Casting Do...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
2801,SI_interpelation_interior_0.txt,988,1003,SI,Najlepša hvala.,False,[],train,[],[],...,[],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
2802,RU_discrediting_the_west_4.txt,244,418,RU,А там намечается самая радикальная перестройка...,False,[],train,[],[],...,[],[None],[None],[The input text discusses a significant restru...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text discusses a si..."
2803,SI_interpelation_interior_7.txt,0,274,SI,"Zanimivo ob tem dejstvu je, da se pa na jugovz...",True,"[Causal_Oversimplification, Appeal_to_Fear-Pre...",train,[],[],...,[None],[None],[None],[The Appeal to Time technique is not used in t...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The Appeal to Time technique ..."


In [11]:
# Assume your DataFrame is called df
label_1 = clean[clean['gold'] == 1]

# Count how many label==1 per category
counts = label_1['prompt_name'].value_counts()

# For label == 0, sample same number of rows per category
label_0_samples = (
    df[df['gold'] == 0]
    .groupby('prompt_name', group_keys=False)
    .apply(lambda x: x.sample(n=1*counts.get(x.name, 0)))
)

# Combine both
balanced_df11 = pd.concat([label_1, label_0_samples])
balanced_df11 = balanced_df11.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df11

/var/folders/lp/5z8ys0rj1q960h2656xp8c680000gn/T/ipykernel_49770/294674448.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=1*counts.get(x.name, 0)))


,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,updated_definition,think_content,illegal_value,explanation,extra_content,source_file,error,status,split,expected_output
0,RU_discrediting_the_west_11.txt,0,125,RU,"Генерала Попова отправили в СИЗО, уволили без ...",False,[],train,[],[],...,[],[None],[None],[The input text does not contain any strong ne...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,RU_migrants_5.txt,354,574,RU,йчас Каменск-Уральск во всех сводках новостей....,True,[Appeal_to_Fear-Prejudice],train,[],[],...,[None],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41_MINI...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
2,pl_abortion_11_04_2024_n05.txt,4987,5641,PL,I jeszcze słowo o hipokrytach i hipokryzji.\nT...,True,"[Conversation_Killer, Straw_Man, Repetition]",train,[],[],...,[None],[None],[None],[The Strawman technique is used in the input t...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The Strawman technique is use..."
3,RU_URW_4.txt,753,934,RU,"Как видим, от администрации Трампа тоже не слы...",True,"[Causal_Oversimplification, Appeal_to_Authority]",train,[],[],...,[None],[None],[None],[Causal Oversimplification is used in the inpu...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,[None],[None],train,"{""description"": ""Causal Oversimplification is ..."
4,pl_abortion_11_04_2024_n05.txt,9681,10556,PL,Dziś jesteśmy w demograficznym kryzysie. Polki...,True,"[Conversation_Killer, Causal_Oversimplificatio...",train,[],[],...,[None],[None],[None],[The Slogans technique is explicitly used in t...,[None],data/llm_output/persuasion/explain/GPT_41_MINI...,NaN,NaN,train,"{""description"": ""The Slogans technique is expl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865,20240704_URW_BG_29.txt,7361,7404,BG,"\nГосподин Запрянов, искате ли да добавите н",False,[],train,[],[],...,[None],[None],[None],[The Appeal to Values technique was not used i...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The Appeal to Values techniqu..."
1866,RU_discrediting_the_west_1.txt,952,1002,RU,Одиноким намного легче манипулировать и управл...,False,[],train,[],[],...,[],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
1867,20231208_URW_BG_71.txt,973,1687,BG,Идеята да отхвърлим ветото на президента е кра...,True,"[Exaggeration-Minimisation, Flag_Waving, Quest...",train,"[Нямам съмнение, че евро-атлантическото мнозин...",[евро-атлантическото мнозинство],...,[Name Calling - Labeling is a form of argument...,[None],[None],[The Name Calling - Labeling technique is used...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The Name Calling - Labeling t..."
1868,20240704_URW_BG_30.txt,4089,4570,BG,на позиция от миналата седмица в Народното съб...,False,[],train,[],[],...,[None],[None],[None],[The input text does not contain any elements ...,[None],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."


In [12]:
SYSTEM_PART_NAME = "system"
USER_PART_NAME = "user"

def build_messages(text,expected_output,job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"):
    prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

    json_line = {
            "messages": [
                {"role": "system", "content": prompt_layout[SYSTEM_PART_NAME].format(technique_name=prompt_content['technique_name'],
                                                                                     technique_desc=prompt_content['technique_desc'],
                                                                                     input_example = prompt_content['input_example'],
                                                                                     output_example = prompt_content['output_example'])},
                {"role": "user", "content": prompt_layout[USER_PART_NAME].format(text = text)},
                {"role": "assistant", "content": expected_output}
            ]
        }
    return json_line
    # return(json.dumps(json_line, ensure_ascii=False) + "\n")


In [13]:
prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

In [14]:
# Write DataFrame to JSONL
ft_df = balanced_df11
with open("data/finetuning/explain2detect_pers_v2_11.jsonl", "w", encoding="utf-8") as file:
    for _, row in ft_df[ft_df.error.isna()].iterrows():
        json_line = build_messages(text = row['text'],expected_output = row['expected_output'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")

In [15]:
# Write DataFrame to JSONL
ft_df = balanced_df21
with open("data/finetuning/explain2detect_pers_v2_21.jsonl", "w", encoding="utf-8") as file:
    for _, row in ft_df[ft_df.error.isna()].iterrows():
        json_line = build_messages(text = row['text'],expected_output = row['expected_output'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")

In [16]:
# Write DataFrame to JSONL
ft_df = balanced_df31
with open("data/finetuning/explain2detect_pers_v2_31.jsonl", "w", encoding="utf-8") as file:
    for _, row in ft_df[ft_df.error.isna()].iterrows():
        json_line = build_messages(text = row['text'],expected_output = row['expected_output'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")

# errors

Unhandled exception during LLM call

In [61]:
errs_to_delete = df[df['error'].apply(lambda x: isinstance(x, np.ndarray) and 'Unhandled exception during LLM call' in x)]
errs_to_delete

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,is_valid,illegal_value,think_content,extra_content,updated_definition,source_file,error,status,split,expected_output


In [91]:
errs_to_delete = df[(df["explanation"].apply(lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [None]))&(df.prompt_name=="Questioning_the_Reputation")].value_counts('source_file').reset_index().head(28)
errs_to_delete

,source_file,count
0,data/llm_output/persuasion/explain/GPT_41/v001...,4
1,data/llm_output/persuasion/explain/GPT_41/v001...,4
2,data/llm_output/persuasion/explain/GPT_41/v001...,4
3,data/llm_output/persuasion/explain/GPT_41/v001...,4
4,data/llm_output/persuasion/explain/GPT_41/v001...,3
5,data/llm_output/persuasion/explain/GPT_41/v001...,3
6,data/llm_output/persuasion/explain/GPT_41/v001...,2
7,data/llm_output/persuasion/explain/GPT_41/v001...,1
8,data/llm_output/persuasion/explain/GPT_41/v001...,1


In [92]:
errs_to_delete = df[(df["explanation"].apply(lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [None]))].value_counts('source_file').reset_index().head(28)
errs_to_delete

,source_file,count
0,data/llm_output/persuasion/explain/GPT_41/v001...,5
1,data/llm_output/persuasion/explain/GPT_41/v001...,4
2,data/llm_output/persuasion/explain/GPT_41/v001...,4
3,data/llm_output/persuasion/explain/GPT_41/v001...,4
4,data/llm_output/persuasion/explain/GPT_41/v001...,4
5,data/llm_output/persuasion/explain/GPT_41/v001...,4
6,data/llm_output/persuasion/explain/GPT_41/v001...,3
7,data/llm_output/persuasion/explain/GPT_41/v001...,3
8,data/llm_output/persuasion/explain/GPT_41/v001...,3
9,data/llm_output/persuasion/explain/GPT_41/v001...,3


In [ ]:
import os

# List of file paths
files_to_delete = errs_to_delete['source_file'].dropna().unique().tolist()

# Delete each file safely
for path in files_to_delete:
    if os.path.isfile(path):
        try:
            # os.remove(path)
            print(f"Deleted: {path}")
        except Exception as e:
            print(f"Error deleting {path}: {e}")
    else:
        print(f"File not found: {path}")

Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/bg/train-bg-part_0003.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/bg/train-bg-part_0001.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/ru/train-ru-part_0009.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/bg/train-bg-part_0000.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/pl/train-pl-part_0006.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/bg/train-bg-part_0002.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/si/train-si-part_0001.parquet
Deleted: data/llm_output/persuasion/explain/GPT_41/v001/Questioning_the_Reputation/train/bg/train-bg-part_0005.parquet
Deleted: data/llm_output/persuasion/explain/GPT_

In [62]:
df[~df.error.isna()]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,is_valid,illegal_value,think_content,extra_content,updated_definition,source_file,error,status,split,expected_output
30,pl_abortion_11_04_2024_n07.txt,3811,4153,PL,"My, kobiety i mężczyźni broniący prawa do życi...",True,[Appeal_to_Values],train,[],[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41_MINI...,[None],[None],train,"{""description"": ""The input text discusses the ..."
31,pl_abortion_11_04_2024_n08.txt,3138,3188,PL,(Poseł Joanna Scheuring-Wielgus: Zejdź z mówni...,True,[Loaded_Language],train,[],[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41_MINI...,[None],[None],train,"{""description"": ""The input text does not prese..."
32,pl_current_affairs_09_01_2025_n02.txt,8069,9438,PL,W decyzji generalny dyrektor ochrony środowisk...,True,"[Appeal_to_Authority, Appeal_to_Values]",train,[],[],...,[True],[None],[None],[None],[None],data/llm_output/persuasion/explain/GPT_41_MINI...,"[{'code': '429', 'message': 'Requests to the C...",[no validation],train,"{""description"": null, ""verdict"": false}"
33,pl_eu_12_06_2024_n02.txt,5000,5586,PL,Poseł Marcin Porzucek:\nPanie Marszałku! Wysok...,True,[Questioning_the_Reputation],train,"[podejmowaliśmy stosowne działania, państwo na...",[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41_MINI...,[None],[None],train,"{""description"": ""The input text does not prese..."
34,pl_eu_12_06_2024_n01.txt,2653,3161,PL,Po prezentacji przez ministrów uzasadnienia pr...,False,[],train,[],[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41_MINI...,[None],[None],train,"{""description"": ""The input text does not prese..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,trail_raw_BG_7.txt,2179,2221,BG,"АНЕ, от място): Я пак!\n\nИСКРЕН МИТЕВ: Това",False,[],train,[],[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
6548,20240704_URW_BG_30.txt,5823,6119,BG,"заседанието, което ще бъде абсолютно безсмисле...",True,"[Consequential_Oversimplification, Exaggeratio...",train,[],[абсолютно безсмислено],...,[True],[None],[None],[None],[This technique consists in making an impressi...,data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not emplo..."
6549,trail_raw_BG_7.txt,2225,2773,BG,"итат от Мария Захарова, която е директно предс...",True,"[Exaggeration-Minimisation, Name_Calling-Label...",train,[],"[проруска партия, проруска партия]",...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not emplo..."
6550,20240229_URW_BG_67.txt,4167,4219,BG,"ПРЕДСЕДАТЕЛ РОСЕН ЖЕЛЯЗКОВ: Времето, госпожо Н...",False,[],train,[],[],...,[True],[None],[None],[None],[],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."


In [ ]:
df[df['raw'].apply(lambda x: isinstance(x, np.ndarray) and None in x)]

In [ ]:
df[~df.error.isna()].value_counts('source_file')